In [20]:
#!pip install hanlp
# !pip install simalign
%load_ext autoreload
%autoreload 2

In [2]:
import os, csv

import pandas as pd
from tqdm import tqdm
from simalign import SentenceAligner
# from google.cloud import translate_v2 as translate

/usr/local/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Below is a working example using the provided data files.
To align Italian and Spanish to English, using the files in the 'second-sense' folder. Otherwise, use the files in the 'Original' folder. 

In [21]:

src_lan = 'en'
# target language
tgt_lan = 'BN'
language = "Bengali"

RESOURCE = '~/Desktop/Research/Sense-Projection'
DATA =  RESOURCE+'/data/'+language
TRANS_FILE = os.path.join(DATA, 'gold-sentences-'+language+'.tsv')
TOKENS_FILE = os.path.join(DATA, language+'-Gold.tsv')
ALIGN_SENT_FILE = os.path.join(DATA, 'sentences-aligned-'+language+'.tsv')
ALIGN_TOKEN_FILE = os.path.join(DATA, 'tokens-aligned-'+language+'.tsv')

# DATA = os.path.join(RESOURCE, 'data')
DATA =  RESOURCE+'/data/English/Original/'

KEY = os.path.join(RESOURCE, 'key')
RAW_SENSE_TSV = os.path.join(DATA, 'gold-tokens-English-wSenses.tsv')
RAW_SENTENCE_TSV = os.path.join(DATA, 'gold-sentences-English.tsv')


# DATA

In [22]:
# raw
raw_sense_df = pd.read_csv(RAW_SENSE_TSV, delimiter='\t', quoting=csv.QUOTE_NONE)
raw_sense_df.head()
print(len(raw_sense_df['Token'].tolist()))

2525


In [23]:
sentence_df = pd.read_csv(RAW_SENTENCE_TSV, delimiter='\t',  quoting=csv.QUOTE_NONE)
sentence_df.head()
sents = sentence_df["ID"].tolist()
len(sents)
# print(sents)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jairiley/Desktop/Research/Sense-Projection/data/English/Original/gold-sentences-English.tsv'

In [14]:
sids = sentence_df["ID"].tolist()
# sids = [i.replace('d', 'i') for i in sids]
# sids[:4]
print(len(sids))

138


In [15]:
trans_df = pd.read_csv(TRANS_FILE, delimiter='\t', quoting=csv.QUOTE_NONE)
trans_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jairiley/Desktop/Research/Sense-Projection/data/Bengali/gold-sentences-Bengali.tsv'

In [17]:
tkns_df = pd.read_csv(TOKENS_FILE, delimiter='\t', quoting=csv.QUOTE_NONE)
tkns_df.head(1000)

,Token ID,Token,Lemma,POS,BN Synset,Gloss
0,d001.s001.t001,এই,এই,X,NaN,NaN
1,d001.s001.t002,নথিটি,নথি,N,bn:00028015n,Writing that provides information (especially ...
2,d001.s001.t003,ইউরোপীয়,ইউরোপ,N,NaN,NaN
3,d001.s001.t004,পাবলিক,পাবলিক,A,bn:00109211a,Not private; open to or concerning the people ...
4,d001.s001.t005,অ্যাসেসমেন্ট,অ্যাসেসমেন্ট,N,bn:00006502n,The act of judging or assessing a person or si...
...,...,...,...,...,...,...
995,d002.s020.t022,গণ্য,গণ্য,V,bn:00095141v,Regard or consider in a specific way
996,d002.s020.t023,হবে,হবে,V,NaN,NaN
997,d002.s020.t024,।,।,X,NaN,NaN
998,d002.s021.t001,:,:,X,NaN,NaN


In [18]:
tgt_tkns = tkns_df['Token'].tolist()
len(tgt_tkns)

2287

# ALIGNMENT

In [19]:
device = 'cpu'
layer = 8
# "bert": "bert-base-multilingual-cased",
# "xlmr": "xlm-roberta-base"
model = 'xlmr'
# mwmf (Match), inter (ArgMax), itermax (IterMax)
mode = 'inter'
matching_methods = 'mai'
token_type='bpe'
aligner = SentenceAligner(
    model=model
    , token_type=token_type
    , matching_methods=matching_methods
    , layer=layer
    , device=device
)

/usr/local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2025-09-09 15:37:24,297 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: xlm-roberta-base


In [10]:
raw_sense_df.head()

,Token ID,Token,Lemma,POS,BN Synset
0,d001.s001.t001,This,NaN,X,NaN
1,d001.s001.t002,document,document,N,bn:00028015n
2,d001.s001.t003,is,be,V,NaN
3,d001.s001.t004,a,NaN,X,NaN
4,d001.s001.t005,summary,summary,N,bn:00075142n


Run the below cell only if using the files in the English/Original directory.

In [10]:

idxes = raw_sense_df['Token ID'].tolist()
tokens = raw_sense_df['Token'].tolist()
en_sents = []
pre_sid = None
s = []
for i, t in zip(idxes, tokens):
    # print(i)
    sid = i.split('.')[1]
    if sid != pre_sid:
        if s:
            en_sents.append(s)
        pre_sid = sid
        s = [t]
    else:
        s.append(t)
en_sents.append(s)


idxes = tkns_df['Token ID'].tolist()
tokens = tkns_df['Token'].tolist()

tgt_sents = []
pre_sid = None
s = []
for i, t in zip(idxes, tokens):
    sid = i.split('.')[1]
    if sid != pre_sid:
        if s:
            tgt_sents.append(s)
        pre_sid = sid
        s = [t]
    else:
        s.append(t)
tgt_sents.append(s)

print(len(en_sents),len(tgt_sents))

NameError: name 'tkns_df' is not defined

Run the below cell only if you are using the files in the English/second-sense directory as well as the provided Spanish or Italian files. 

In [ ]:
idxes = raw_sense_df['Token ID'].tolist()
tokens = raw_sense_df['Token'].tolist()
# # print(idxes,tokens)
en_sents = []
pre_sid = None
sent = []
for i, s in zip(idxes, tokens):
    if s != "-end-":
        sent.append(s)
    else:
        en_sents.append(sent)
        sent = []


idxes = tkns_df['Token ID'].tolist()
tokens = tkns_df['Token'].tolist()

tgt_sents = []
pre_sid = None
sent = []
for i, s in zip(idxes, tokens):
    if s != "-end-":
        sent.append(s)
    else:
        tgt_sents.append(sent)
        sent = []
print(len(en_sents),len(tgt_sents))

121
[['En', 'el', 'presente', 'documento', 'se', 'resume', 'el', 'Informe', 'Público', 'Europeo', 'de', 'Evaluación', '(', 'EPAR', ')', '.'], ['En', 'él', 'se', 'explica', 'cómo', 'el', 'Comité', 'de', 'Medicamentos', 'de', 'Uso', 'Humano', '(', 'CHMP', ')', 'ha', 'evaluado', 'los', 'estudios', 'realizados', 'con', 'el', 'medicamento', 'a fin de', 'emitir', 'unas', 'recomendaciones', 'sobre', 'su', 'uso', '.'], ['Si', 'desea', 'más', 'información', 'sobre', 'su', 'enfermedad', 'o', 'el', 'tratamiento', 'de', 'la', 'misma', ',', 'le', 'aconsejamos', 'que', 'lea', 'el', 'prospecto', '(', 'incluido', 'en', 'el', 'EPAR', ')', 'o', 'pregunte', 'a', 'su', 'médico', 'o', 'su', 'farmacéutico', '.'], ['Si', 'desea', 'más', 'información', 'sobre', 'el', 'fundamento', 'en', 'el', 'que', 'se', 'han', 'basado', 'las', 'recomendaciones', 'del', 'CHMP', ',', 'le', 'aconsejamos', 'que', 'lea', 'el', 'Debate', 'Científico', '(', 'también', 'incluido', 'en', 'el', 'EPAR', ')', '.'], ['¿Qué', 'es', 'Alim

In [30]:
print(en_sents[0])
print(tgt_sents[0])

['This', 'document', 'is', 'a', 'summary', 'of', 'the', 'European', 'Public', 'Assessment', 'Report', '(', 'EPAR', ')', '.']
['En', 'el', 'presente', 'documento', 'se', 'resume', 'el', 'Informe', 'Público', 'Europeo', 'de', 'Evaluación', '(', 'EPAR', ')', '.']


In [32]:
# do the alignment
aligns = []
for en_s, tgt_s in zip(tqdm(en_sents), tgt_sents):
    # print(en_s,tgt_s,'\n')
    alignments = aligner.get_word_aligns(en_s, tgt_s)
    align = ''
    for w1, w2 in alignments[mode]:
        align += f'{w1}-{w2} '
    align = align.strip()
    aligns.append(align)

100%|██████████| 121/121 [01:25<00:00,  1.41it/s]


In [ ]:
aligns[0]

'0-0 4-9 7-2 8-3 10-5 11-6 12-7 13-8 14-9 14-10'

In [33]:
print(len(aligns))
trans_df['English-'+language+' Alignment'] = aligns
trans_df.head(200)

121


,ID,Sentence,English-Spanish Alignment
0,d000.s000,En el presente documento se resume el Informe ...,0-2 1-3 2-4 6-6 7-9 8-8 9-11 10-7 11-12 12-13 ...
1,d000.s001,En él se explica cómo el Comité de Medicamento...,0-1 1-3 2-4 3-5 4-6 5-7 6-8 6-11 7-8 8-9 9-11 ...
2,d000.s002,Si desea más información sobre su enfermedad o...,0-0 2-1 3-2 4-3 5-4 6-5 8-6 9-7 11-9 12-13 13-...
3,d000.s003,Si desea más información sobre el fundamento e...,0-0 2-1 3-2 4-3 5-4 6-5 7-6 10-16 11-14 12-17 ...
4,d000.s004,¿Qué es Alimta ?,0-0 1-1 2-2 3-3
...,...,...,...
116,d000.s116,El tratamiento del vómito debe realizarse siem...,0-20 2-0 3-1 4-2 5-3 6-4 7-6 8-5 9-8 10-9 11-1...
117,d000.s117,¿Cuáles son los efectos secundarios de Cerenia ?,0-0 1-1 2-2 3-3 5-5 6-6 7-7
118,d000.s118,"En general , Cerenia fue bien tolerado cuando ...",0-2 1-3 2-0 3-4 4-5 5-6 6-8 7-9 8-10 9-11 11-1...
119,d000.s119,¿Qué precauciones debe adoptar la persona que ...,0-0 3-1 5-4 6-5 7-6 8-7 9-8 10-9 11-10 12-11 1...


In [ ]:
trans_df.to_csv(ALIGN_SENT_FILE, sep='\t', index=False)

# ALIGN TOKENS IN TOKEN FILE

In [34]:
tkns_df.head()

,Token ID,Token,Lemma,POS,BN Synset
0,NaN,En,NaN,X,NaN
1,NaN,el,NaN,X,NaN
2,d000.s000.t000,presente,presente,ADJ,bn:00108977a
3,d000.s000.t001,documento,documento,NOUN,bn:00028015n;bn:00028017n
4,NaN,se,NaN,X,NaN


In [35]:
aligned_tks_df = pd.read_csv(RAW_SENSE_TSV, delimiter='\t', quoting=csv.QUOTE_NONE)
aligned_tks_df.head()

,Token ID,Token,Lemma,POS,BN Synset
0,NaN,This,NaN,X,NaN
1,d000.s000.t000,document,document,NOUN,bn:00028015n;wiki:document;wn:document%1:10:00...
2,NaN,is,be,VERB,NaN
3,NaN,a,NaN,X,NaN
4,d000.s000.t001,summary,summary,NOUN,bn:00075142n;wiki:summary_(law);wn:sum-up%1:10...


In [36]:
aligned_tks_df['Token ID '+tgt_lan] = None
aligned_tks_df['Token '+tgt_lan] = None
aligned_tks_df['Lemma '+tgt_lan] = None
aligned_tks_df['POS '+tgt_lan] = None
aligned_tks_df['Sense '+tgt_lan] = None

In [37]:
aligned_tks_df.head()
print(len(aligned_tks_df['Token']))

2759


In [38]:
trans_df.head()

,ID,Sentence,English-Spanish Alignment
0,d000.s000,En el presente documento se resume el Informe ...,0-2 1-3 2-4 6-6 7-9 8-8 9-11 10-7 11-12 12-13 ...
1,d000.s001,En él se explica cómo el Comité de Medicamento...,0-1 1-3 2-4 3-5 4-6 5-7 6-8 6-11 7-8 8-9 9-11 ...
2,d000.s002,Si desea más información sobre su enfermedad o...,0-0 2-1 3-2 4-3 5-4 6-5 8-6 9-7 11-9 12-13 13-...
3,d000.s003,Si desea más información sobre el fundamento e...,0-0 2-1 3-2 4-3 5-4 6-5 7-6 10-16 11-14 12-17 ...
4,d000.s004,¿Qué es Alimta ?,0-0 1-1 2-2 3-3


In [39]:
def process_nan(s):
  if type(s)!=type(''):
    return ''
  return s

Run the below alignment if using the English/Original files.

In [40]:
count_en = 0
sentence_num = 0
count_es = 0
for i, row in trans_df.iterrows():
    # print(row.iloc[0])
    sent_i = row.iloc[0]
    # print(sent_i)
    # print(sent_i)
    tgt_sent_i = sent_i
    # print(row)
    # sent_i = sent_i.replace("i","d")
    # tgt_sent = tkns_df.loc[count_es:count_es+len(tgt_sents[sentence_num])]
    # en_sent = aligned_tks_df.loc[count_en:count_en+len(en_sents[sentence_num])]
    # print(tgt_sent.loc[2])
    # print(row)
    for alignment in row['English-'+language+' Alignment'].split():
        try:
            w1, w2 = alignment.split('-')
            
            en_id = f'{sent_i}.t{int(w1) + 1:03d}'
            tgt_id = f'{tgt_sent_i}.t{int(w2) + 1:03d}'
            row_en = aligned_tks_df[aligned_tks_df['Token ID'] == en_id]
            row_tgt = tkns_df[tkns_df['Token ID'] == tgt_id]
            # print(row_tgt)
    
            # print(aligned_tks_df['Token ID'] )
            # print(row_tgt)
            # print(row_en.iloc[0]['Token ID '+tgt_lan])
            if row_en.iloc[0]['Token ID '+tgt_lan] is not None:
                aligned_tks_df.loc[row_en.index, 'Token ID '+tgt_lan] += '➕' + row_tgt.iloc[0]['Token ID']
                aligned_tks_df.loc[row_en.index, 'Token '+tgt_lan] += '➕' + row_tgt.iloc[0]['Token']
                if aligned_tks_df.loc[row_en.index, 'POS '+tgt_lan].item() != process_nan(row_tgt.iloc[0]['POS']):
                    aligned_tks_df.loc[row_en.index, 'POS '+tgt_lan] += '➕' + process_nan(row_tgt.iloc[0]['POS'])
                if aligned_tks_df.loc[row_en.index, 'Sense '+tgt_lan].item() != process_nan(row_en.iloc[0]['BN Synset']):
                    aligned_tks_df.loc[row_en.index, 'Sense '+tgt_lan] += '➕' + process_nan(str(row_en.iloc[0]['BN Synset']).split(";")[0])
            else:
                # print(str(row_en.iloc[0]['BN Synset']).split(";")[0])
                # print(row_tgt.iloc[0]['Token ID'])
                # print(row_en.index.values)
                aligned_tks_df.loc[row_en.index.values[0], 'Token ID '+tgt_lan] = row_tgt.iloc[0]['Token ID']
                # print(aligned_tks_df.loc[row_en.index, 'Token ID '+tgt_lan],row_tgt.iloc[0]['Token ID'],'\\n\\n')
        
                aligned_tks_df.loc[row_en.index.values[0], 'Token '+tgt_lan] = row_tgt.iloc[0]['Token']
                aligned_tks_df.loc[row_en.index.values[0], 'Lemma '+tgt_lan] = row_tgt.iloc[0]['Lemma']
                aligned_tks_df.loc[row_en.index.values[0], 'POS '+tgt_lan] = row_tgt.iloc[0]['POS']
                aligned_tks_df.loc[row_en.index.values[0], 'Sense '+tgt_lan] = str(row_en.iloc[0]['BN Synset']).split(";")[0]
                aligned_tks_df.loc[row_en.index.values[0], 'BN Synset'] = str(row_en.iloc[0]['BN Synset']).split(";")[0]
        except:
            print(f"error {i}")
    # count_en += len(en_sents[sentence_num])
    # count_es += len(tgt_sents[sentence_num])
    # sentence_num += 1


error 0
error 0
error 0
error 0
error 0
error 0
error 0
error 0
error 0
error 1
error 1
error 1
error 1
error 1
error 1
error 1
error 1
error 1
error 1
error 1
error 1
error 1
error 1
error 1
error 1
error 1
error 1
error 2
error 2
error 2
error 2
error 2
error 2
error 2
error 2
error 2
error 2
error 2
error 2
error 2
error 2
error 2
error 2
error 3
error 3
error 3
error 3
error 3
error 3
error 3
error 3
error 3
error 3
error 3
error 3
error 4
error 4
error 4
error 4
error 5
error 5
error 5
error 5
error 5
error 5
error 5
error 5
error 5
error 5
error 5
error 5
error 6
error 6
error 6
error 6
error 7
error 7
error 7
error 7
error 8
error 8
error 8
error 8
error 8
error 8
error 8
error 8
error 8
error 8
error 8
error 8
error 8
error 8
error 8
error 8
error 8
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9
error 9


Run the below for alignment if you are using the provided English/second-sense files and the Spanish or Italian provided files. 

In [42]:
count_en = 0
sentence_num = 0
count_es = 0
for i, row in trans_df.iterrows():
    # print(row.iloc[0])
    sent_i = row.iloc[0]
    # print(sent_i)
    # print(sent_i)
    tgt_sent_i = sent_i
    # print(row)
    # sent_i = sent_i.replace("i","d")
    tgt_sent = tkns_df.loc[count_es:count_es+len(tgt_sents[sentence_num])]
    en_sent = aligned_tks_df.loc[count_en:count_en+len(en_sents[sentence_num])]
    # print(tgt_sent.loc[2])
    # print(row)
    for alignment in row['English-'+language+' Alignment'].split():
        w1, w2 = alignment.split('-')
        row_tgt = tgt_sent.loc[count_es+int(w2)]
        row_en = en_sent.loc[count_en+int(w1)]
        if row_en['Token ID '+tgt_lan] is not None:
            if str(aligned_tks_df.loc[count_en+int(w1), 'Token ID '+tgt_lan]) != "nan":
                aligned_tks_df.loc[count_en+int(w1), 'Token ID '+tgt_lan] += '➕' + str(row_tgt['Token ID'])
            else:
                aligned_tks_df.loc[count_en+int(w1), 'Token ID '+tgt_lan] = str(row_tgt['Token ID'])
            aligned_tks_df.loc[count_en+int(w1), 'Token '+tgt_lan] += '➕' + row_tgt['Token']
            if aligned_tks_df.loc[count_en+int(w1), 'POS '+tgt_lan] != process_nan(row_tgt['POS']):
                aligned_tks_df.loc[count_en+int(w1), 'POS '+tgt_lan] += '➕' + process_nan(row_tgt['POS'])
            if aligned_tks_df.loc[count_en+int(w1), 'Sense '+tgt_lan] != process_nan(row_en['BN Synset']):
                if str(aligned_tks_df.loc[count_en+int(w1), 'Sense '+tgt_lan]) != "nan":
                    aligned_tks_df.loc[count_en+int(w1), 'Sense '+tgt_lan] += '➕' + process_nan(row_en['BN Synset'])
        else:
            aligned_tks_df.loc[count_en+int(w1), 'Token ID '+tgt_lan] = row_tgt['Token ID']
            aligned_tks_df.loc[count_en+int(w1), 'Token '+tgt_lan] = row_tgt['Token']
            aligned_tks_df.loc[count_en+int(w1), 'Lemma '+tgt_lan] = row_tgt["Lemma"]
            aligned_tks_df.loc[count_en+int(w1), 'POS '+tgt_lan] = row_tgt['POS']
            aligned_tks_df.loc[count_en+int(w1), 'Sense '+tgt_lan] = row_en['BN Synset']

    count_en += len(en_sents[sentence_num])
    count_es += len(tgt_sents[sentence_num])
    sentence_num += 1
    # print(count_en)

In [43]:
# aligned_tks_df[]
aligned_tks_df.head(2000)

,Token ID,Token,Lemma,POS,BN Synset,Token ID ES,Token ES,Lemma ES,POS ES,Sense ES
0,NaN,This,NaN,X,NaN,d000.s000.t000,presente,presente,ADJ,NaN
1,d000.s000.t000,document,document,NOUN,bn:00028015n;wiki:document;wn:document%1:10:00...,d000.s000.t001,documento,documento,NOUN,bn:00028015n;wiki:document;wn:document%1:10:00...
2,NaN,is,be,VERB,NaN,NaN,se,NaN,X,NaN
3,NaN,a,NaN,X,NaN,None,None,None,None,None
4,d000.s000.t001,summary,summary,NOUN,bn:00075142n,d000.s000.t003,Informe,informe,NOUN,bn:00075142n
...,...,...,...,...,...,...,...,...,...,...
1995,NaN,-end-,NaN,X,NaN,d000.s086.t005,caso,caso,NOUN,NaN
1996,NaN,For,NaN,X,NaN,NaN,detallado,detallar,VERB,NaN
1997,NaN,each,NaN,X,NaN,None,None,None,None,None
1998,d000.s086.t000,country,country,NOUN,bn:00011771n;bn:00023235n;wiki:country;wiki:so...,NaN,estos,NaN,X,bn:00011771n;bn:00023235n;wiki:country;wiki:so...


In [49]:
aligned_tks_df.to_csv(ALIGN_TOKEN_FILE, sep='\t', index=False)